In [1]:
import os
import numpy as np
import pandas as pd
os.chdir("/content/drive/MyDrive/cs289/proj/code")

In [2]:
pip install pysolnp

     |████████████████████████████████| 914 kB 5.2 MB/s 


In [3]:
"""
Test 1: data process
"""
from data_process import data_prep_task
X_t, X_v, Y_t, Y_v = data_prep_task('wash', test_size=0.3)

Categorical features imputed successfully.
Discrete numerical features imputed successfully.
Int values turned into float type successfully.
Continuous numerical features imputed successfully.
Dataset is standardized.
Data pre-processing is finished.


In [4]:
X_t = X_t[:, 0:16]
X_t = X_t.astype(float)

In [5]:
"""
Test 2: learners
"""
from learners import *
from sl_task import sl_task

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [6]:
# define task
task = sl_task(X = X_t, Y = Y_t, family = "Gaussian", K = 10)

# define candidate learners
L_glm = Lrnr_glm(sl_task = task)
L_lasso = Lrnr_glmnet(sl_task = task)
L_rf = Lrnr_rf(sl_task = task)
L_xgb = Lrnr_xgboost(sl_task = task, num_round = 200)

stack = [L_glm, L_lasso, L_rf, L_xgb]

In [7]:
# test learners individually
# - glm
L_glm.train(Y = Y_t, X = X_t)
y_hat = L_glm.predict(X = X_t)
mse1 = square_error_loss(y_hat, Y_t)

# - glmnet
L_lasso.train(Y = Y_t, X = X_t)
y_hat = L_lasso.predict(X = X_t)
mse2 = square_error_loss(y_hat, Y_t)

# - randomforest
L_rf.train(Y = Y_t, X = X_t)
y_hat = L_rf.predict(X = X_t)
mse3 = square_error_loss(y_hat, Y_t)

# - xgboost
L_xgb.train(Y = Y_t, X = X_t)
y_hat = L_xgb.predict(X = X_t)
mse4 = square_error_loss(y_hat, Y_t)

In [8]:
# test discrete sl
sl = initialize_sl(sl_task = task, stack = stack, meta = "solnp")

In [9]:
sl.train()

Training on Fold 1 of 10 

Training on Fold 2 of 10 

Training on Fold 3 of 10 

Training on Fold 4 of 10 

Training on Fold 5 of 10 

Training on Fold 6 of 10 

Training on Fold 7 of 10 

Training on Fold 8 of 10 

Training on Fold 9 of 10 

Training on Fold 10 of 10 



In [10]:
# summary sl
sl.summary()

 Num. of cv folds:  10 
 Meta learner:  solnp 
 n:  3286 
 p:  16
  learner        weight   cv_risk
0     glm  2.487788e-02  1.371883
1  glmnet  1.827999e-08  1.370490
2      rf  8.008017e-01  1.024441
3     xgb  1.743204e-01  1.051700
